In [ ]:


from flask import request, Response
from flask import Flask
from config import netbox_api
from jinja2 import Environment, FileSystemLoader
from nornir_napalm.plugins.tasks import napalm_get, napalm_cli
import pprint
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
from nornir_jinja2.plugins.tasks import template_file
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
import re

from credentials import(netbox_url,
                        netbox_token,
                        device_username,
                        device_password)
from nornir import InitNornir
from nornir.core.filter import F

In [ ]:
def create_nornir_session():
    """ 
    Инициализируем nornir, но для "hosts" используем данные из netbox
    :return: nr_session
    """
    nr_session = InitNornir(
        inventory={
            "plugin": "NetBoxInventory2",
            "options": {
                "nb_url": netbox_url,
                "nb_token": netbox_token,
                "group_file": "./inventory/groups.yml",
                "defaults_file": "./inventory/defaults.yml",
            },
        },
    )
    return nr_session

In [ ]:
def conversion(tup, dict = {}):
    for x, y in tup:
        dict.setdefault(x, []).append(y)
    return dict

In [ ]:
templates_path = "./templates/"

In [ ]:
def ios_config_interfaces(task: Task) -> Result:
    """ 
    Основная задача: конфигурация интерфейса
    """
    ios_interface_template = task.run(  # Подзадача: получаем шаблон
        name = 'Get the configuration template, fixed value...',
        task = template_file, # функция, импортированная из "nornir_jinja2.plugins.tasks"
        template = 'cisco_ios_interface.template', 
        path = templates_path 
    )
    """ task.run( # Подзадача: отправляем готовый конфиг на устройство
        name = 'Complete the configuration, fixed value through Netmiko',
        task = netmiko_send_config, # функция, импортированная из "nornir_netmiko.tasks"
        config_commands = ios_interface_template.result.split('\n'),
        cmd_verify = True
    ) """
    #print(ios_interface_template)
    
    return Result(
        host=task.host,
        result=ios_interface_template
    )

In [ ]:
#device_type = netbox_api.dcim.device_types.get(slug='cisco_ios').id
#device_type

""" def hello_world(task: Task) -> Result:
    
    
    return Result(
        host=task.host,
        result=f"{task.host.name} says hello world!"
    ) """


def push_config_interface(netbox_interface):
    
    filter_query = '10.30.1.105'
    nr = create_nornir_session()
    sw = nr.filter(hostname = filter_query) # производим отбор по конкретному хосту
    #print(sw.inventory.hosts)

    get_int = sw.run(task=napalm_get, getters=['get_interfaces']) # получаем все интерфейсы с устройства в виде словаря
    #print_result(get_interfaces)
    for device in get_int.values():
        interfaces = device.result['get_interfaces'].keys() # получаем интерфейсы как ключи словаря
        if netbox_interface in (intf for intf in list(interfaces)):
            print("Find {} for device {}".format(netbox_interface, device.host))
    
    #result = sw.run(netmiko_send_command, command_string="show version")
    #result_a = sw.run(task=hello_world)
    #print_result(result_a)
    
    """ result = sw.run(
        name="Configuration interface.../",
        task=ios_config_interfaces)
    print_result(result) """
            
    return Response(status=204)
    
push_config_interface('Ethernet0/2')


In [221]:
def delete_config_intf(netbox_interface):
    
    print("Delete interface {} config...".format(netbox_interface))

In [222]:
def create_config_intf(netbox_interface):
    
    print("Push new interface {} config...".format(netbox_interface))

In [223]:
def update_config_intf(netbox_interface):
    
    print("Updating interface {} config...".format(netbox_interface))

In [224]:
def manage_device_interfaces():
    
    devices_keys = ['role','device_id','intf_id'] # список ключей для словаря devices
    devices = [] 
    templates_roles = ['access_switch', 'user_device'] # получаем из netbox (произвольные данные)
    device_roles = []
    regex = "[a|b]_terminations"
    
    get_device_cable = conversion(list(netbox_api.dcim.cables.get(request.json["data"]["id"])))
    #get_device_cable = conversion(list(netbox_api.dcim.cables.get('306')))

    for key in get_device_cable.keys(): # заполняем список device_value и объединяем с device_keys в словарь
        if re.match(regex, key): # отбираем нужные ключи из словаря по регулярке
            devices_values = []
            device_id = get_device_cable[key][0][0]['device']['id']
            devices_values.append(netbox_api.dcim.devices.get(device_id).device_role.slug) # роль устройства
            devices_values.append(device_id) # id устройства
            devices_values.append(get_device_cable[key][0][0]['id']) # id интерфейса устройства
            devices.append(dict(zip(devices_keys,devices_values))) # получаем список из словарей
            print("Dictionary append into list...")
    
    for device in devices: # заполняем список ролей
        device_roles.append(device['role'])
     
    if set(device_roles) == set(templates_roles): # проверяем, что получили устройства с разными ролями и в соответствии со списком    
        for device in devices:
            if device['role'] == templates_roles[0]: # нам нужен access switch
                device_intf_id = device['intf_id'] # получаем ID интерфейса access switchа из, нами созданного, словаря            
        get_device_interface = netbox_api.dcim.interfaces.get(device_intf_id) # по ID находим интерфейс в netbox    
        print("List is equal {}, switch access interface ID: {}...".format(device_roles, device_intf_id))
        if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсов
            print("\tManagement interface, no changes will be performed...")
        else:
            if request.json["event"] == "deleted": # Конфиг интерфейса адрес будет удален

                delete_config_intf( netbox_interface=get_device_interface
                                    )

            elif request.json["event"] == "created": # Конфиг интерфейса будет добавлен

                create_config_intf( netbox_interface=get_device_interface)

            elif request.json["event"] == "updated": # Конфиг интерфейса будет изменен

                update_config_intf( netbox_interface=get_device_interface
                                    )
         
    else: print("List is not equal")
    
        
    return Response(status=204)
#manage_device_interfaces()

In [225]:
# Create a Flask instance
app = Flask(__name__)
app.add_url_rule("/api/config_intf",
                methods=['POST'],
                view_func=manage_device_interfaces)
    
if __name__ == "__main__": 
    app.run(host='0.0.0.0', port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.96.5.16:8080
Press CTRL+C to quit


Dictionary append into list...
Dictionary append into list...
List is equal ['access_switch', 'user_device'], switch access interface ID: 136...
Updating interface Ethernet0/3 config...


10.30.1.226 - - [18/Mar/2023 21:59:24] "POST /api/config_intf HTTP/1.1" 204 -
